In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# !pip install tensorflow-hub
# !pip install tensorflow-datasets

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
print(tf.__version__)

2.15.0


In [ ]:
# train_data, validation_data, test_data = tfds.load(
#     name="imdb_reviews",
#     split=('train[:60%]', 'train[60%:]', 'test'),
#     as_supervised=True)

In [ ]:
# embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
# hub_layer = hub.KerasLayer(embedding, input_shape=[],
#                            dtype=tf.string, trainable=True)
# # hub_layer(train_examples_batch[:3])

In [ ]:
# (X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data()

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/IMDB Dataset.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
print(df.iloc[0, 0])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [ ]:
print(max(df.iloc[:, 0].str.split(" ").map(len)))

2470


In [ ]:
X = df.iloc[:, 0]
y = df.iloc[:, 1]

In [ ]:
y.shape

(50000,)

In [ ]:
encoder = OrdinalEncoder(dtype = np.int32)
y = encoder.fit_transform(np.expand_dims(y, axis = 1))

In [ ]:
encoder.categories_

[array(['negative', 'positive'], dtype=object)]

In [ ]:
print(type(y))

<class 'numpy.ndarray'>


In [ ]:
print(y.shape)

(50000, 1)


In [ ]:
y = np.squeeze(y, axis = 1)

In [ ]:
print(y.shape)

(50000,)


In [ ]:
print(set(y))

{0, 1}


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 47, stratify = y)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(40000,)
(10000,)
(40000,)
(10000,)


In [ ]:
# X_train[0]

In [ ]:
# X_train = tf.convert_to_tensor(X_train)

In [ ]:
max_features = 10000
sequence_length = 2048

vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens = max_features,
    pad_to_max_tokens = True,
    output_sequence_length = sequence_length
)
vectorize_layer.adapt(X_train)

In [ ]:
lst = vectorize_layer.get_vocabulary()

In [ ]:
lst[:10]

['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it']

In [ ]:
lst[0] = "[PAD]"
lst.insert(2, "[START]")
lst.insert(3, "[END]")

In [ ]:
lst[:10]

['[PAD]', '[UNK]', '[START]', '[END]', 'the', 'and', 'a', 'of', 'to', 'is']

In [ ]:
with open("vocab.txt", 'w') as f:
    for idx, word in enumerate(lst):
        f.write(word)
        f.write(" " + str(idx) + "\n")

In [ ]:
lst[-1:-10:-1]

['lex',
 'lifts',
 'loop',
 'martino',
 'mcdermott',
 'menu',
 'momma',
 'monstrous',
 'mount']

In [ ]:
X_train = vectorize_layer(X_train)
X_test = vectorize_layer(X_test)

In [ ]:
X_train.shape

TensorShape([40000, 2048])

In [ ]:
embedding_dim = 512
input = tf.keras.layers.Input(shape = (sequence_length,))
# x = vectorize_layer(input)
x = tf.keras.layers.Embedding(max_features, embedding_dim)(input)
# x = tf.keras.layers.Embedding(max_features, embedding_dim)(x)
# x = hub_layer(x)
# x = tf.keras.layers.GlobalAveragePooling1D()(x)
# x = tf.keras.layers.Dense(32, activation = "relu", kernel_regularizer = tf.keras.regularizers.L2())(x)
# x = tf.keras.layers.Dropout(0.5)(x)
# output = tf.keras.layers.Dense(3, activation = 'softmax')(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True, activation = 'tanh', kernel_regularizer = tf.keras.regularizers.L2()))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, activation = 'tanh', kernel_regularizer = tf.keras.regularizers.L2()))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(64, activation='tanh', kernel_regularizer = tf.keras.regularizers.L2())(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(32, activation='tanh', kernel_regularizer = tf.keras.regularizers.L2())(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
model = tf.keras.Model(inputs = input, outputs = output)


# model = tf.keras.Sequential(
#     [
#         vectorize_layer,
#         # tf.keras.layers.Embedding(max_features, embedding_dim, input_length = sequence_length),
#         tf.keras.layers.Dense(128, activation = "relu"),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.GlobalAveragePooling1D(),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.Dense(3, activation = 'softmax')
#     ]
# )


In [ ]:
# model = tf.keras.Sequential()

In [ ]:
# model.add(vectorize_layer)
# model.add(hub_layer)
# # model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True, activation = 'tanh', kernel_regularizer = tf.keras.regularizers.L2())))
# model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation = 'tanh', kernel_regularizer = tf.keras.regularizers.L2())))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Dense(64, activation='tanh', kernel_regularizer = tf.keras.regularizers.L2()))
# model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2048)]            0         
                                                                 
 embedding (Embedding)       (None, 2048, 512)         5120000   
                                                                 
 bidirectional (Bidirection  (None, 2048, 128)         295424    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                             

In [ ]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = ["accuracy"]
)

In [ ]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 875s 3s/step - loss: 12.2153 - accuracy: 0.4965


[12.215311050415039, 0.4964999854564667]

In [ ]:
history = model.fit(
    X_train, y_train, batch_size = 128, epochs = 1, validation_data = (X_test, y_test),
)

233/313 [=====================>........] - ETA: 40:41 - loss: 2.2809 - accuracy: 0.7730

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


In [ ]:
y_pred = model.predict(X_test) > 0.5

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, confusion_matrix

# Compute F1 score
f1 = f1_score(y_test, y_pred)

# Compute precision
precision = precision_score(y_test, y_pred)

# Compute recall
recall = recall_score(y_test, y_pred)

# Generate classification report
report = classification_report(y_test, y_pred)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(conf_matrix)


In [ ]:
data = [
    ["This product is ok."],
    ["Dissapointed with this."]
]
data = vevectorize_layer(data)
y_pred = model.predict(data)
print(y_pred)
y_pred = np.argmax(y_pred, axis = 1)

for text, prediction in zip(data, y_pred):
    print(f"{text[0]} - {encoder.categories_[0][prediction]}")

In [ ]:
encoder.categories_[0]

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)
